In [1]:
import pandas as pd
import datetime as dt
# Load the revenue, cost, and cohort size data

path = 'C:/Users/sydne/Desktop/Business Analyst business case __ MACADAM.xlsx'

revenue_data = pd.read_excel(path, sheet_name='raw_revenue_cohorts')
cost_data = pd.read_excel(path, sheet_name='raw_cost_cohorts')
cohort_data = pd.read_excel(path, sheet_name='raw_cohorts_size')

#checking all date related columns only have the 1st date, therefore in the next merging part
#we could directly merge dataframes using the cohort month column.
def date_check(df, column):
    
    cohort_dates = set(pd.to_datetime(df[column], format='%Y-%m-%d').dt.day)
    
    if len(cohort_dates) == 1:
        print('True')
    else:
        print('False')

date_check(revenue_data,'cohort_month')
date_check(revenue_data,'returning_date')
date_check(cost_data,'cohort_month')
date_check(cost_data,'returning_month')
date_check(cohort_data,'month_of_creation')

True
True
True
True
True


In [2]:
#adding a column with same column name for merging later
revenue_data['returning_month'] = revenue_data['returning_date']  
cohort_data['cohort_month'] = cohort_data['month_of_creation']


# Merge revenue and cost data, only matching the periods which both revenue and cost records exist
cohort_revenue_cost = pd.merge(revenue_data, cost_data, on=['cohort_month', 'returning_month'])
final_df = pd.merge(cohort_revenue_cost, cohort_data, on=['cohort_month'])

#calculate the life span of each record by unit of days
final_df['period'] = (final_df['returning_month']-final_df['cohort_month']).dt.days
# if it's within the same month, substitute the life span with at least one day
final_df['period'].replace(0, 1, inplace=True)

#calculate the numbers needed to come up with customer life time value
#customer life time value = average customer value * average life span
#= (total revenue - total cost)/total cohort size * average life span per user

#calculate the revenue and cost produced by each cohort in a single day
final_df['daily_revenue'] = final_df['revenue']/final_df['period']
final_df['daily_cost'] = final_df['cost']/final_df['period']

#calculate the total revenue and cost produced by all the cohorts in a single day
total_revenue = final_df['daily_revenue'].sum()
total_cost = final_df['daily_cost'].sum()

#calculate the total cohort size
total_cohort_size = cohort_data['cohort_size'].sum()

#calculate the average days each user stay in the app by each cohort month

avg_days =  final_df.groupby('cohort_month')['period'].sum()/final_df.groupby('cohort_month')['cohort_size'].max()
# calculate the total days each user stay in the app
avg_days = avg_days.mean()



In [3]:
final_df

,cohort_month,returning_date,revenue,returning_month,cost,month_of_creation,cohort_size,period,daily_revenue,daily_cost
0,2022-07-01,2022-08-01,27,2022-08-01,-1.6,2022-07-01,112,31,0.870968,-0.051613
1,2022-07-01,2022-09-01,4,2022-09-01,-1.6,2022-07-01,112,62,0.064516,-0.025806
2,2022-07-01,2023-03-01,11,2023-03-01,-38.4,2022-07-01,112,243,0.045267,-0.158025
3,2022-07-01,2023-04-01,8,2023-04-01,-13.6,2022-07-01,112,274,0.029197,-0.049635
4,2022-08-01,2022-08-01,118,2022-08-01,-2.4,2022-08-01,1490,1,118.000000,-2.400000
5,2022-08-01,2022-09-01,59,2022-09-01,-6.4,2022-08-01,1490,31,1.903226,-0.206452
6,2022-08-01,2022-10-01,19,2022-10-01,-6.4,2022-08-01,1490,61,0.311475,-0.104918
7,2022-08-01,2022-11-01,34,2022-11-01,-26.4,2022-08-01,1490,92,0.369565,-0.286957
8,2022-08-01,2022-12-01,47,2022-12-01,-55.2,2022-08-01,1490,122,0.385246,-0.452459
9,2022-08-01,2023-01-01,61,2023-01-01,-110.4,2022-08-01,1490,153,0.398693,-0.721569


In [4]:
#calculate final customer life time value
LTV = ((total_revenue + total_cost)/total_cohort_size)*avg_days
LTV

0.06867889769360526

<h1> Reflection </h1>
In the end, we came up with a very small customer life time value, because we are using an over estimated cohort size. In reality, the cohort each month will have large amounts of overlaps and the most accurate way is to count the number of unique ids in the total cohort pool. Then we should get a more accurate ltv.